In [2]:
# import patool
# Download CBDB sqlite
# !git clone https://github.com/cbdb-project/cbdb_sqlite.git
import requests
import os
import warnings
import urllib3

# 禁用所有 InsecureRequestWarning 警告
warnings.simplefilter('ignore', urllib3.exceptions.InsecureRequestWarning)

# url = "https://hf-mirror.com/datasets/cbdb/cbdb-sqlite/resolve/main/latest.7z?download=true"
url = "https://huggingface.co/datasets/cbdb/cbdb-sqlite/resolve/main/latest.7z?download=true"
r = requests.get(url, verify=False)
os.makedirs("./cbdb_sqlite", exist_ok=True)
with open("./cbdb_sqlite/latest.7z", "wb") as f:
    f.write(r.content)

KeyboardInterrupt: 

In [3]:
# import glob
import os
latest_file = "./cbdb_sqlite/latest.7z"
print(latest_file)
latest_db = latest_file.split("/")[-1].split(".")[0] + ".db"
print(latest_db)
if os.path.isfile(latest_db):
    os.remove(latest_db)

./cbdb_sqlite/latest.7z
latest.db


In [15]:
# # Explode latest database file
# 打印 patool 的安装路径
# from pyunpack import Archive
# Archive(latest_file).extractall(".")

ValueError: patool not found! Please install patool!

In [20]:
import py7zr

# 指定压缩文件和目标目录
archive = './cbdb_sqlite/latest.7z'
output_dir = ''

# 解压文件
with py7zr.SevenZipFile(archive, 'r') as archive:
    archive.extractall(output_dir)

print("解压完成！")



解压完成！


In [3]:

import sqlite3
import pandas as pd
import os
global CONN
latest_file = "./cbdb_sqlite/latest.7z"
database_file = os.path.basename(latest_file).split(".")[0]+".db"
CONN = sqlite3.connect(database_file)

In [4]:
tables = ['BIOG_MAIN', 'ASSOC_DATA', 'BIOG_ADDR_DATA', 'ALTNAME_DATA', 'KIN_DATA', 'ENTRY_DATA', 'POSTED_TO_OFFICE_DATA', 'STATUS_DATA', 'TEXT_CODES']  
sql_str = ''    
for table in tables:  
    sql_str += f"SELECT '{table}' AS TableName, COUNT(*) AS RowCount FROM {table} UNION ALL "    
# 去除最后一个 UNION ALL  
sql_str = sql_str[:-11]
print(sql_str)
# sql_str 


SELECT 'BIOG_MAIN' AS TableName, COUNT(*) AS RowCount FROM BIOG_MAIN UNION ALL SELECT 'ASSOC_DATA' AS TableName, COUNT(*) AS RowCount FROM ASSOC_DATA UNION ALL SELECT 'BIOG_ADDR_DATA' AS TableName, COUNT(*) AS RowCount FROM BIOG_ADDR_DATA UNION ALL SELECT 'ALTNAME_DATA' AS TableName, COUNT(*) AS RowCount FROM ALTNAME_DATA UNION ALL SELECT 'KIN_DATA' AS TableName, COUNT(*) AS RowCount FROM KIN_DATA UNION ALL SELECT 'ENTRY_DATA' AS TableName, COUNT(*) AS RowCount FROM ENTRY_DATA UNION ALL SELECT 'POSTED_TO_OFFICE_DATA' AS TableName, COUNT(*) AS RowCount FROM POSTED_TO_OFFICE_DATA UNION ALL SELECT 'STATUS_DATA' AS TableName, COUNT(*) AS RowCount FROM STATUS_DATA UNION ALL SELECT 'TEXT_CODES' AS TableName, COUNT(*) AS RowCount FROM TEXT_CODES


In [5]:
cursor = CONN.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("数据库中的表：", cursor.fetchall())


数据库中的表： [('ADDR_BELONGS_DATA',), ('ADDR_CODES',), ('ADDR_PLACE_DATA',), ('ADDR_XY',), ('ADDRESSES',), ('ALTNAME_CODES',), ('ALTNAME_DATA',), ('APPOINTMENT_TYPE_CODES',), ('ASSOC_CODE_TYPE_REL',), ('ASSOC_CODES',), ('ASSOC_DATA',), ('ASSOC_TYPES',), ('ASSUME_OFFICE_CODES',), ('BIOG_ADDR_CODES',), ('BIOG_ADDR_DATA',), ('BIOG_INST_CODES',), ('BIOG_INST_DATA',), ('BIOG_MAIN',), ('BIOG_SOURCE_DATA',), ('BIOG_TEXT_DATA',), ('CBDB_NAME_LIST',), ('CHORONYM_CODES',), ('CopyMissingTables',), ('CopyTables',), ('CopyTablesDefault',), ('COUNTRY_CODES',), ('DYNASTIES',), ('ENTRY_CODE_TYPE_REL',), ('ENTRY_CODES',), ('ENTRY_DATA',), ('ENTRY_TYPES',), ('ETHNICITY_TRIBE_CODES',), ('EVENT_CODES',), ('EVENTS_ADDR',), ('EVENTS_DATA',), ('EXTANT_CODES',), ('ForeignKeys',), ('FormLabels',), ('GANZHI_CODES',), ('HOUSEHOLD_STATUS_CODES',), ('INDEXYEAR_TYPE_CODES',), ('KIN_DATA',), ('KIN_Mourning',), ('KIN_MOURNING_STEPS',), ('KINSHIP_CODES',), ('LITERARYGENRE_CODES',), ('MEASURE_CODES',), ('NIAN_HAO',), ('OCCA

In [6]:
tablecount_df = pd.read_sql_query(sql_str, CONN).drop_duplicates().dropna()
tablecount_df

,TableName,RowCount
0,BIOG_MAIN,641568
1,ASSOC_DATA,187878
2,BIOG_ADDR_DATA,440059
3,ALTNAME_DATA,196850
4,KIN_DATA,540631
5,ENTRY_DATA,253437
6,POSTED_TO_OFFICE_DATA,1976515
7,STATUS_DATA,60912
8,TEXT_CODES,57387


In [7]:
datatypes = ["Number of Persons 人數","Social Associations 社會關係", "Biographical Addresses 地址", "Alternate Names 別名", "Kin Relationships 親屬關係", "Entry into Office 入仕", "Office Postings 任官", "Social Distinction 社會區分", "Texts 著作"]
tablecount_df['TableName'] = tablecount_df['TableName'].replace(dict(zip(tablecount_df['TableName'], datatypes)))  
tablecount_df = tablecount_df.rename(columns={'TableName': "DataTypes"})
tablecount_df 

,DataTypes,RowCount
0,Number of Persons 人數,641568
1,Social Associations 社會關係,187878
2,Biographical Addresses 地址,440059
3,Alternate Names 別名,196850
4,Kin Relationships 親屬關係,540631
5,Entry into Office 入仕,253437
6,Office Postings 任官,1976515
7,Social Distinction 社會區分,60912
8,Texts 著作,57387


In [8]:
tablecount_df.to_csv('data_types.csv', index=False)

In [9]:
# Read the content of old JavaScript files
with open('script.js', 'r') as file:  
    js_code = file.read()  
  
# Delete old data  
start_delimiter = 'const chart2Data = ['  
end_delimiter = '];// Chart 2 Data'
start_index = js_code.find(start_delimiter)  
end_index = js_code.find(end_delimiter) + len(end_delimiter)  
js_code = js_code[:start_index] + js_code[end_index:]
# new data
js_data = ',\n'.join(  
    f'{{ label: "{dyname}", value: {count} }}'  
    for dyname,count in zip(tablecount_df["DataTypes"],tablecount_df["RowCount"])  
)
new_data = f'{start_delimiter}\n{js_data}\n{end_delimiter}'  
  
js_code = new_data + "\n" +js_code  
  
# Write updated JavaScript file 
with open('script.js', 'w') as file:  
    file.write(js_code)

In [12]:
# update the date
import datetime  
import re 

svg_path = os.path.join(os.getcwd(), 'images', 'svg4.svg')  
with open(svg_path, 'r',encoding='utf-8') as file:  
    svg_content = file.read()  
  
# get current date  
current_time = datetime.datetime.now().strftime('%Y年%m月%d日') 
current_time_english = datetime.datetime.now().strftime('%B %d, %Y') 
  
updated_svg_content = re.sub(r'更新日期：\d+年\d+月\d+日', f'更新日期：{current_time}', svg_content)    
updated_svg_content = re.sub(r'Updated on: \w+ \d+, \d+', f'Updated on: {current_time_english}', updated_svg_content)
with open(svg_path, 'w',encoding='utf-8') as file:  
    file.write(updated_svg_content) 

In [13]:
# close the db
CONN.close()